In [1]:
import tensorflow
import tensorflow_hub as hub
import numpy as np
import tensorflow_text
from clean_text import clean_text

2022-01-07 12:36:26.065344: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-07 12:36:26.065371: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Text representation byWord Embeddings Transformers -> Universal Sentence Encoder

In [2]:
# load pretrained model
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

2022-01-07 12:36:38.653680: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-01-07 12:36:38.653716: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-07 12:36:38.653743: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (dac-Latitude-E7450): /proc/driver/nvidia/version does not exist
2022-01-07 12:36:38.654046: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
import string
import re
import preprocessor as p
from spacy.lang.en import stop_words as spacy_stopwords  # we use spacy's list of stop words to clean our data

stop_words = spacy_stopwords.STOP_WORDS
punctuations = string.punctuation

def clean(text):
    
    text = re.sub(r'\W+', ' ', text)  # remove non-alphanumeric characters
    # replace numbers with the word 'number'
    text = re.sub(r"\d+", "number", text)
    # don't consider sentenced with less than 3 words (i.e. assumed noise)
    if len(text.strip().split()) < 3:
        return None
    text = text.lower()  # lower case everything
    
    return text.strip() # remove redundant spaces

In [5]:
from sklearn.datasets import fetch_20newsgroups
# Prepare data

corpus = fetch_20newsgroups()
categories = fetch_20newsgroups()["target_names"]
train_set = fetch_20newsgroups(subset = 'train', categories = categories)
test_set = fetch_20newsgroups(subset = 'test', categories = categories)

X_train = train_set.data
y_train = train_set.target

X_test = test_set.data
y_test = test_set.target


sample = 35

# show sample of train_set
print("LABEL: {}".format(categories[y_train[sample]]))
print(X_train[sample])
print(clean(X_train[sample]))

LABEL: rec.sport.hockey
From: dchhabra@stpl.ists.ca (Deepak Chhabra)
Subject: Re: Goalie masks
Nntp-Posting-Host: stpl.ists.ca
Organization: Solar Terresterial Physics Laboratory, ISTS
Lines: 21

In article <120666@netnews.upenn.edu> kkeller@mail.sas.upenn.edu (Keith Keller) writes:
>My vote goes to John Vanbiesbrouck.  His mask has a skyline of New York
>City, and on the sides there are a bunch of bees (Beezer).  It looks
>really sharp.

Funny you should mention this; one time on HNIC Don Cherry pointed out
Vanbiesbrouck's mask.  He _hated_ it.  I think he said something to the effect
of:
"You see?  He was great last year; now he goes out and gets that dopey mask 
and he can't stop a beachball!"

You may or may not take Cherry seriously at all, but I cracked up when I heard
it.

I think Ed Belfour has the current best mask in the NHL btw.  I also like
Moog's, and I'll give Fuhr's new one an honourable mention, although I haven't
seen it closely yet (it looked good from a distance!).  

In [23]:
# find th header
sample = 1
pattern = "(From)(.|\n)*(: )(\d)*(\n)"
replace = ""
res = re.search(pattern, X_train[sample])
# print(X_train[sample])
new = re.sub(pattern, replace, X_train[sample])
print(new)

NNTP-Posting-Host: carson.u.washington.edu

A fair number of brave souls who upgraded their SI clock oscillator have
shared their experiences for this poll. Please send a brief message detailing
your experiences with the procedure. Top speed attained, CPU rated speed,
add on cards and adapters, heat sinks, hour of usage per day, floppy disk
functionality with 800 and 1.4 m floppies are especially requested.

I will be summarizing in the next two days, so please add to the network
knowledge base if you have done the clock upgrade and haven't answered this
poll. Thanks.

Guy Kuo <guykuo@u.washington.edu>



https://www.nbshare.io/notebook/197284676/Word-Embeddings-Transformers-In-SVM-Classifier-Using-Python/ -> clue